# Advent of Code 2018 - Day 3

In [1]:
data = []
with open('inputs_day_3.txt', 'r') as f:
  for line in f:
    data.append(line.strip())

data[:5]

['#1 @ 167,777: 23x12',
 '#2 @ 253,106: 10x25',
 '#3 @ 104,622: 11x25',
 '#4 @ 267,61: 14x16',
 '#5 @ 829,831: 19x10']

In [2]:
# Parse
claims = []
for line in data:

  id = int(line[line.find('#') + 1 : line.find(' @')])
  loc = [int(x) for x in line[line.find('@') + 2 : line.find(':')].split(',')]
  size = [int(x) for x in line[line.find(':') + 2: ].split('x')]
  claims.append((id, tuple(loc), tuple(size)))

claims[:10]

[(1, (167, 777), (23, 12)),
 (2, (253, 106), (10, 25)),
 (3, (104, 622), (11, 25)),
 (4, (267, 61), (14, 16)),
 (5, (829, 831), (19, 10)),
 (6, (886, 374), (22, 12)),
 (7, (129, 812), (18, 17)),
 (8, (972, 677), (14, 17)),
 (9, (123, 249), (12, 14)),
 (10, (669, 330), (11, 21))]

## Part 1

Lazy solution: just make the darn grid and keep count. The description says that the whole piece of fabric is at least $1000$ inches, but it seems that no rectangle is defined outside the a $1000 \times 1000$ fabric with its top corner at $(0, 0)$.

I have hard-coded the grid size, but it can be easily deduced by finding the rectangle farthest bottom corner from the origin (or something along those lines).

Many optimizations can be made, but an easy one is use a hashmap instead of grid. This way we only worry about points inside the rectangles.

In [3]:
size = 1000
grid = [[0] * size for i in range(size)]

#print(*grid, sep='\n')

for claim in claims:
  column, row = claim[1]
  width, height = claim[2]

  for i in range(row, row + height):
    for j in range(column, column + width):
      grid[i][j] += 1

  #print(*grid, sep='\n')
  #print()


sum([sum([x >= 2 for x in row]) for row in grid])

111935

## Part 2

Lazy solution using Part 1: Start with the grid from the end of Part 1, go over all claims again. If a claim corresponds to a rectangle of all $1$'s, it is the rectangle that does not overlap with any other.

In [4]:
for claim in claims:
  column, row = claim[1]
  width, height = claim[2]

  intact = True
  for i in range(row, row + height):
    for j in range(column, column + width):
      if grid[i][j] != 1:
        intact = False
        break

  if intact:
    print(claim)
    break
  

(650, (830, 151), (25, 21))


A more general solution: Find the rectangle that does not overlap with any of the others without drawing the grid.

In [5]:
def overlap(l1, r1, l2, r2): 
      
    # If one rectangle is on left side of other 
    if(l1[0] >= r2[0] or l2[0] >= r1[0]):
      return False
  
    # If one rectangle is above other (modified condition: since up is down)
    if(l1[1] >= r2[1] or l2[1] >= r1[1]): 
      return False
  
    return True

overlap((1, 3), (5, 7), (3, 1), (7, 5))

True

The obvious code ([slightly modified from GeeksforGeeks](https://www.geeksforgeeks.org/find-two-rectangles-overlap/)) defines a rectangle by top left and bottom right points. All rectangles will need to be expressed in this form.

l1: Top Left coordinate of first rectangle.\
r1: Bottom Right coordinate of first rectangle.\
l2: Top Left coordinate of second rectangle.\
r2: Bottom Right coordinate of second rectangle.

In our input, the first coorindate of a rectangel is given, and we can find the second by simple addition.



In [6]:
for a in claims:
  l1 = a[1]
  r1 = (a[2][0] + l1[0], a[2][1] + l1[1])

  overlap_counter = 0
  for b in claims:
    if a != b:
      l2 = b[1]
      r2 = (b[2][0] + l2[0], b[2][1] + l2[1])
      overlap_counter += overlap(l1, r1, l2, r2) # Can actually break if overlap_counter == 1 after increment since 1 is enough to disqualify

  if overlap_counter == 0:
    print(a)
    break # Since we are told, only one exists


(650, (830, 151), (25, 21))
